<a href="https://colab.research.google.com/github/jhmuller/nextWord/blob/main/nextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
import string
import numpy as np
import sklearn

https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf

In [2]:
URL = 'https://www.gutenberg.org/files/67138/67138-0.txt'
response = requests.get(URL)
orig_text = response.content

In [3]:
M = 2440
temp_text = str(orig_text[M:])
clean_text = re.sub("\\\\r|\\\\n|\\\\xbb|\\\\xef|\\\\xbf|'b'", " ", temp_text)
cleaner_text = clean_text.translate(str.maketrans('', '', string.punctuation))

In [4]:
len(cleaner_text)
cleaner_text[:200]

'bBOOK I                                          CHAPTER                                     1      Robert Cohn was once middleweight boxing champion of Princeton Do not  think that I am very much imp'

In [51]:
 
N = len(cleaner_text)
cutoff = int(np.floor(0.5*N))
while cleaner_text[cutoff] != ' ':
  cutoff += 1
train = cleaner_text[:cutoff]
test = cleaner_text[cutoff:]
test[:100]

' out of the cages one at  a time and they have steers in the corral to receive them and keep them  f'

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts([train])

def make_Xy(data, tokenizer, ngram_len=1):

  sequence_data = tokenizer.texts_to_sequences([data])[0]
  print(len(sequence_data))
  sequences = []
  X = []
  y = []
  for i in range(1, len(sequence_data)-ngram_len-1):
      words = sequence_data[i-1:i+ngram_len]
      X.append(words[:-1])
      y.append(words[-1])

  X = np.array(X)
  y = np.array(y)

  from  tensorflow.keras.utils import to_categorical
  vocab_size = len(tokenizer.word_index) + 1

  ycat = to_categorical(y, num_classes=vocab_size)
  return X, ycat, y

Xtrain, ytrain, yindstrain = make_Xy(train, tokenizer, ngram_len=3)
Xtest, ytest, yindstest = make_Xy(test, tokenizer, ngram_len=3)



34953
31254


In [76]:
print(tokenizer.sequences_to_texts([yindstest[:5]]))
tokenizer.sequences_to_texts(Xtest[:5])

['one at a time and']


['out of the', 'of the one', 'the one at', 'one at a', 'at a time']

In [53]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
def get_vocab_size(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([data])
  vocab_size = len(tokenizer.word_index) + 1
  return vocab_size

train_vocab_size = get_vocab_size(train)
model = Sequential()
model.add(Embedding(train_vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(train_vocab_size, activation="softmax"))

In [54]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [55]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer=Adam(lr=0.001))
model.fit(Xtrain, ytrain, epochs=20, batch_size=64, callbacks=[reduce_lr, tensorboard_Visualization])

Epoch 1/20
547/547 [==============================] - 11s 14ms/step - loss: 6.5957 - accuracy: 0.0564 - lr: 0.0010
Epoch 2/20
547/547 [==============================] - 8s 14ms/step - loss: 6.1312 - accuracy: 0.0726 - lr: 0.0010
Epoch 3/20
547/547 [==============================] - 8s 14ms/step - loss: 5.7280 - accuracy: 0.0962 - lr: 0.0010
Epoch 4/20
547/547 [==============================] - 8s 14ms/step - loss: 5.3740 - accuracy: 0.1164 - lr: 0.0010
Epoch 5/20
547/547 [==============================] - 8s 14ms/step - loss: 5.0910 - accuracy: 0.1311 - lr: 0.0010
Epoch 6/20
547/547 [==============================] - 8s 15ms/step - loss: 4.8341 - accuracy: 0.1453 - lr: 0.0010
Epoch 7/20
547/547 [==============================] - 8s 15ms/step - loss: 4.5935 - accuracy: 0.1576 - lr: 0.0010
Epoch 8/20
547/547 [==============================] - 8s 15ms/step - loss: 4.3460 - accuracy: 0.1678 - lr: 0.0010
Epoch 9/20
547/547 [==============================] - 8s 15ms/step - loss: 4.0925 - acc

In [72]:
vpreds = model.predict(Xtrain[:10])
print(len(vpreds))

1/1 [==============================] - 0s 57ms/step
10


In [68]:
vpreds

array([[3.1657301e-17, 7.0367020e-07, 3.9614457e-05, ..., 4.4670700e-24,
        2.2778228e-19, 1.1904125e-23],
       [1.1373858e-20, 4.3945145e-03, 1.3321155e-04, ..., 2.7161548e-25,
        2.4375866e-27, 6.0519539e-30],
       [2.4143169e-19, 4.4415381e-05, 2.1865437e-04, ..., 4.4792042e-22,
        8.2951495e-31, 2.1465009e-28],
       ...,
       [6.9978097e-17, 7.4614718e-06, 1.7154634e-05, ..., 2.9582267e-31,
        6.1213479e-23, 4.7216100e-31],
       [9.0251609e-19, 2.3083126e-02, 1.3680741e-01, ..., 1.3861023e-28,
        1.8005581e-27, 1.6768800e-24],
       [4.6138811e-19, 3.2106396e-02, 1.0949762e-03, ..., 3.2657033e-15,
        2.1778165e-17, 4.2019396e-23]], dtype=float32)

In [69]:
pred_inds = [np.argmax(x) for x in vpreds]
pred_words = tokenizer.sequences_to_texts([pred_inds])
pred_words

['1 robert cohn was waiting middleweight boxing champion we princeton']

In [70]:
true_inds = [np.argmax(x) for x in ytrain[:10]]
true_words = tokenizer.sequences_to_texts([true_inds])
true_words

['1 robert cohn was once middleweight boxing champion of princeton']

In [71]:
tokenizer.sequences_to_texts(Xtrain[:20])

['bbook i chapter',
 'i chapter 1',
 'chapter 1 robert',
 '1 robert cohn',
 'robert cohn was',
 'cohn was once',
 'was once middleweight',
 'once middleweight boxing',
 'middleweight boxing champion',
 'boxing champion of',
 'champion of princeton',
 'of princeton do',
 'princeton do not',
 'do not think',
 'not think that',
 'think that i',
 'that i am',
 'i am very',
 'am very much',
 'very much impressed']